In [1]:
import warnings
import pandas as pd
import numpy as np
import nltk
import collections
import random
# import matplotlib.pyplot as plt
# import seaborn as sns
# import os
# import re
# import math
# from sqlalchemy import create_engine
# import pymysql
# import requests
# from bs4 import BeautifulSoup
# from sklearn.linear_model import LinearRegression
# from statsmodels.tsa.ar_model import AR
# from statsmodels.tools.eval_measures import rmse
# from scipy import stats

In [2]:
pd.set_option('display.max_columns', 100)
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('./clean_data/clean_data.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)

### Step 1: prepare dataset to be used for identifying ingredients and measures
- each cocktail = one row
- punches to be excluded due to scaling issues
- rows related to garnish to be excluded
- all ingredients generalized and combined in one column

In [4]:
df.head()

,strDrink,strCategory,strGlass,strIngredients,Alc_type,Basic_taste,strInstructions,strMeasures,Value,MeasureName,Value_numeric,Value_ml,Value_gr,Garnish_amount,Garnish_type,MeasureName_copy
0,'57 Chevy with a White License Plate,Cocktail,Highball glass,Creme De Cacao White,Creamy Liqueur,NaN,1. Fill a rocks glass with ice 2.add white cre...,1 oz white,1,oz,1.0,30.0,NaN,NaN,NaN,white
1,1-900-FUK-MEUP,Shot,Old-fashioned glass,Absolut Kurant,Vodka,NaN,Shake ingredients in a mixing tin filled with ...,1/2 oz,1/2,oz,0.5,15.0,NaN,NaN,NaN,oz
2,110 in the shade,Beer,Beer Glass,Lager,Beer,NaN,Drop shooter in glass. Fill with beer,16 oz,16,oz,16.0,480.0,NaN,NaN,NaN,oz
3,151 Florida Bushwacker,Milk / Float / Shake,Beer mug,Malibu Rum,Rum,NaN,Combine all ingredients. Blend until smooth. G...,1/2 oz,1/2,oz,0.5,15.0,NaN,NaN,NaN,oz
4,155 Belmont,Cocktail,White wine glass,Dark Rum,Rum,NaN,Blend with ice. Serve in a wine glass. Garnish...,1 shot,1,shot,1.0,25.0,NaN,NaN,NaN,shot


In [5]:
df_processed = df.copy()     # This way we will always have access to our original dataset

In [7]:
# Drop rows with garnish, this part of the cocktail will be processed on the later stages

df_processed.drop(df_processed.loc[~df_processed['Garnish_type'].isnull()].index, axis=0, inplace=True)

In [9]:
# Drop rows with related to punches to avoid scaling issue in next steps

df_processed.drop(df_processed.loc[df_processed['strCategory'].str.contains('Punch')].index, axis=0, inplace=True)

In [13]:
# Drop nulls in values to avoid ingredients without measures in generated cocktail

df_processed.drop(df_processed.loc[(df_processed['Value_ml'].isnull()) & (df_processed['Value_gr'].isnull())].index,
                 axis=0, inplace=True)

In [18]:
# Combine non-alcoholic ingredients as they are and generalised alcoholic ingredients in one column

df_processed['Ingredients'] = np.where(~df_processed['Basic_taste'].isnull(), df_processed['strIngredients'], np.nan)
df_processed['Ingredients'] = np.where(df_processed['Ingredients'].isnull(),
                                       df_processed['Alc_type'], df_processed['Ingredients'])

In [21]:
# Replace ' ' with '_' to keep adjectives with nouns

df_processed['Ingredients'] = df_processed['Ingredients'].apply(lambda x: x.replace(' ', '_'))

In [26]:
# Define reduced datasets containing only the columns required for particular step

df_ingredients = df_processed[['strDrink', 'Ingredients']]
df_measures = df_processed[['strDrink', 'Ingredients', 'Value_ml', 'Value_gr']]

In [28]:
df_ingredients.head()

,strDrink,Ingredients
0,'57 Chevy with a White License Plate,Creamy_Liqueur
1,1-900-FUK-MEUP,Vodka
2,110 in the shade,Beer
3,151 Florida Bushwacker,Rum
4,155 Belmont,Rum


In [27]:
df_measures.head()

,strDrink,Ingredients,Value_ml,Value_gr
0,'57 Chevy with a White License Plate,Creamy_Liqueur,30.0,NaN
1,1-900-FUK-MEUP,Vodka,15.0,NaN
2,110 in the shade,Beer,480.0,NaN
3,151 Florida Bushwacker,Rum,15.0,NaN
4,155 Belmont,Rum,25.0,NaN


### Step 1: transform dataset:
- each cocktail = one row
- all ingredients generalized and combined in one column

In [ ]:
# # Add non-alcoholic ingredients in a separate column

# df['non_alc_ingredients'] = np.where(~df['Basic_taste'].isnull(), df['strIngredients'], np.nan)

In [ ]:
# # Reduce dataset to only the columns required for current step

# df_ingredients = df[['strDrink', 'strCategory', 'Alc_type', 'non_alc_ingredients', 'Garnish_type']]

In [ ]:
# # Drop rows with garnish, this part of the cocktail will be processed on the later stages

# df_ingredients.drop(df_ingredients.loc[~df_ingredients['Garnish_type'].isnull()].index, axis=0, inplace=True)

In [ ]:
# # Drop rows with related to punches to avoid scaling issue in next steps

# df_ingredients.drop(df_ingredients.loc[df_ingredients['strCategory'].str.contains('Punch')].index, axis=0, inplace=True)

In [ ]:
# # Replace nan values with empty string to align data type for every value

# df_ingredients.fillna('', inplace=True)

In [ ]:
# # Replace ' ' with '_' to keep adjectives with nouns

# df_ingredients['Alc_type'] = df_ingredients['Alc_type'].apply(lambda x: x.replace(' ', '_'))
# df_ingredients['non_alc_ingredients'] = df_ingredients['non_alc_ingredients'].apply(lambda x: x.replace(' ', '_'))

In [ ]:
# Reallocate ingredients as columns so each column takes only one row

df_ingredients = df_ingredients[df_ingredients['Alc_type'] != 'Alc_type']
s =  df_ingredients.groupby('strDrink').cumcount().add(1)
df_ingredients = (df_ingredients.set_index(['strDrink',s])
        .unstack()
        .sort_index(axis=1, level=1)
       )
df_ingredients.columns = ['{}_{}'.format(a, b) for a,b in df_ingredients.columns]

df_ingredients = df_ingredients.reset_index()
df_ingredients.head()

In [ ]:
df_ingredients['Ingredients'] = df_ingredients.drop(['strDrink'], axis=1).fillna('').apply(lambda x:
                                                                          ' '.join(x), axis=1).apply(lambda x: x.lower())
df_ingredients = df_ingredients[['strDrink', 'Ingredients']]

In [ ]:
df_ingredients.head()

### Step 2:
- split ingerients by pairs
- compute most common pairs (N - to be defined depending on results)

In [ ]:
# Generate pairs from ingredients of each cocktail and combine them in one list

bigram = [list(nltk.bigrams(nltk.word_tokenize(i))) for i in df_ingredients['Ingredients']]
pairs_list = [j for i in bigram for j in i]
print(len(pairs_list))
pairs_list[:10]

There are some tuples where elements are swopped but essential taste of such combination is not unique. Such tuples need to be alighned.

In [ ]:
# First, define a list of tuples to be amended

to_aligh = []
for a in pairs_list:
    for b in pairs_list:
        if a != b:
            if b[1] == a[0]:
                if b[0] == a[1]:            # If both elements of tuple are equal to the swopped tuple under the check
                    if a not in to_aligh and b not in to_aligh:
                        to_aligh.append(b)

In [ ]:
# Second, amend them

pairs_list_aligned = []
for i in pairs_list:
    if i in to_aligh:
        pairs_list_aligned.append((i[1], i[0]))
    else:
        pairs_list_aligned.append(i)

In [ ]:
# Check that total number of tuples hasn't changed, only content should be amended

len(pairs_list_aligned) == len(pairs_list)

In [ ]:
# Define 25% of the most common pairs as a separate list

counter=collections.Counter(pairs_list_aligned)
print(len(counter))
common_pairs = counter.most_common(int(len(counter)*0.25))
common_pairs

In [ ]:
common_ingredients = []

for n in common_pairs:
    common_ingredients.append(n[0][0])
    common_ingredients.append(n[0][1])
common_ingredients = list(set(common_ingredients))
common_ingredients

### Step 3: generate a frame for new combinations:
- define total number of ingerients (random choice from a range 3-6))
- from ingredients included in top pairs pick one randomly
- find suitable pair for this ingredient (from all pairs, but give top pairs bigger weight)
- do the same for the next ingredient but check that it's not included already
- etc until limit is reached

In [ ]:
# Define total number of ingredients for a new cocktail

n_ingr = random.randint(3,7)

In [ ]:
new_cocktail = []

In [ ]:
# Pick the first ingredient randomly from the top popular

first_ingr = random.choice(common_ingredients)
new_cocktail.append(first_ingr)

In [ ]:
# Identify remaning ingredients

counter = 1          # Not zero because the first ingredient is defined separatelly above
while counter < n_ingr:
    temp_list = []
    for i in set(pairs_list_aligned):
        if new_cocktail[-1] in i:
            temp_list.append(i)
    random_pair = random.choice(temp_list)
    if random_pair[0] == new_cocktail[-1]:
        next_ingr = random_pair[1]
    else:
        next_ingr = random_pair[0]
    new_cocktail.append(next_ingr)
    counter += 1

In [ ]:
new_cocktail

### Step 4: define volume of each ingredient

In [ ]:
# Create new dataframe with relevant columns only

df_measures = df[['strDrink', 'strCategory', 'Alc_type', 'non_alc_ingredients', 'Value_ml', 'Value_gr', 'Garnish_type']]

In [ ]:
# # Drop rows with garnish, this part of the cocktail will be processed on the later stages

# df_measures.drop(df_measures.loc[~df_measures['Garnish_type'].isnull()].index, axis=0, inplace=True)

In [ ]:
# # Remove cocktails given in big volumes to align scale

# df_measures.drop(df_measures.loc[df_measures['strCategory'].str.contains('Punch')].index, axis=0, inplace=True)

In [ ]:
# Combine values in one column and define measure for each in a separate column

df_measures['Measure'] = np.where(df_measures['Value_ml'].isnull(), 'gr', 'ml')
df_measures['Value'] = df_measures['Value_ml'].fillna(0) + df_measures['Value_gr'].fillna(0)
# df_measures['Ingredients'] = df_measures['Alc_type'].fillna('') + df_measures['non_alc_ingredients'].fillna('')
# df_measures['Ingredients'] = df_measures['Ingredients'].apply(lambda x: x.lower()).apply(lambda x: x.replace(' ', '_'))
df_measures.head()

In [ ]:
# Combine value and measure in one string, this way it will be easier to pick up random value together with correct measure

df_measures['Value'] = df_measures['Value'].astype('object').apply(lambda x: str(x))
df_measures['Value_Measure'] = df_measures[['Value', 'Measure']].apply(lambda x: ' '.join(x), axis=1)
df_measures.head()

In [ ]:
# Pick up random volume of each ingredient

for i in new_cocktail:
    volume = [random.choice(df_measures.loc[df_measures['Ingredients'] == i]['Value_Measure'].tolist()) for i in new_cocktail]
    new_cocktail_final = pd.DataFrame({'Ingredient': new_cocktail, 'Volume': volume})
    new_cocktail_final

### Step 5: define garnish

***1) Prepare list of main ingredients with garnish for each cocktail in dataset***

In [ ]:
# Define reduced dataframe which contains only relevant fields

df_garnish = df[['strDrink', 'strIngredients', 'Alc_type', 'Value_ml', 'Garnish_amount', 'Garnish_type']]

In [ ]:
df_garnish['Ingredients'] = np.where(df_garnish['Alc_type'].isnull(), df_garnish['strIngredients'], df_garnish['Alc_type'])

In [ ]:
# Combine value and measure of garnish in one column so it will be easier to pick it up later in the code

df_garnish['Garnish_ingr'] = np.where(~df_garnish['Garnish_amount'].isnull(), df_garnish['strIngredients'], np.nan)
df_garnish['Garnish'] = df_garnish[['Garnish_ingr', 'Garnish_amount', 'Garnish_type']].fillna('').apply(lambda x:
                                                                                                       ' '.join(x), axis=1)
df_garnish['Garnish'] = df_garnish['Garnish'].apply(lambda x: x.replace('0 top', 'top'))
df_garnish['Garnish'] = np.where(df_garnish['Garnish'] == '  ', np.nan, df_garnish['Garnish'])

In [ ]:
# Rearrange dataframe that way that it's possible to identify garnish per cocktail per dominant ingredient

drink_name_list = []
garnish_list = []
ingredient_list = []

for drink in df_garnish['strDrink'].unique():      # For each cocktial
    df_selected = df_garnish.loc[df_garnish['strDrink'] == drink]
    max_value = df_selected['Value_ml'].max()

    for ingr in df_selected.loc[df_selected['Value_ml'] == max_value]['Ingredients']:
        for garnish in df_selected['Garnish'].unique():
            drink_name_list.append(drink)
            garnish_list.append(garnish)           # Include all garnishes for the drink
            ingredient_list.append(ingr)           # Include an ingredient taking the biggest part of the drink

df_garnish_final = pd.DataFrame({'Drink':drink_name_list, 'Ingredient':ingredient_list, 'Garnish':garnish_list})
df_garnish_final.drop(df_garnish_final.loc[df_garnish_final['Garnish'].isnull()].index, axis=0, inplace = True)

In [ ]:
df_garnish_final.head()

***2) Identify a main ingredient of a generated cocktail***

In [ ]:
new_cocktail_final

In [ ]:
new_cocktail_final = new_cocktail_final.join(new_cocktail_final['Volume'].str.rsplit(n=1, expand=True).rename(columns={
                                                                                                    0: 'Value', 1: 'Measure'}))
new_cocktail_final['Value'] = new_cocktail_final['Value'].astype('float')

In [ ]:
main_ingr = random.choice(new_cocktail_final.loc[new_cocktail_final['Value'] ==
                                                                     new_cocktail_final['Value'].max()]['Ingredient'].tolist())

In [ ]:
new_cocktail_final.drop(['Value', 'Measure'], axis=1, inplace=True)

***3) Find suitable garnish and add to the recipe***

In [ ]:
garnish_to_add = df_garnish_final.loc[df_garnish_final['Ingredient'] == main_ingr]['Garnish'].tolist()

if garnish_to_add:
    new_cocktail_final.loc[len(new_cocktail_final)] = ['garnish', garnish_to_add]

In [ ]:
new_cocktail_final

### List alcoholic ingredients

In [ ]:
df.head()

In [ ]:
df['Alc_type'].unique()

In [ ]:
for i in df['Alc_type'].unique():
    print(i)
    print(df.loc[df['Alc_type'] == i]['strIngredients'].unique())
    print('*'*100)

In [ ]:
# for i in new_cocktail:
#     random.choice(df.loc[df['Alc_type'] == i]['strIngredients'].unique().tolist())

In [ ]:
new_cocktail_final

In [ ]:
a = df_measures.loc[df_measures['Ingredients'] == 'gin']
a.loc[a['Measure'] == 'gr']

In [ ]:
df_measures.loc[df_measures['Value'] == '0.0']

In [ ]:
len(df_measures)

In [ ]:
df.loc[df['strIngredients'] == "Lavender"]

In [ ]:
# Try to set up lists of liqueurs and maybe other alcohol ingredients.

In [ ]:
# Refactoring might be done to rearrange the code that way that to generate new cocktail you don't have to run whole file